# Day 27 Visualization Walkthrough

This notebook demonstrates the reusable plotting functions introduced in `visualization.py` and summarises how to interpret the resulting charts.

## 1. Import helpers and load the dataset
The helper functions encapsulate the Matplotlib/Seaborn configuration used in the lesson scripts.

In [ ]:
from Day_27_Visualization.visualization import (
    load_sales_data,
    build_revenue_by_region_plot,
    build_daily_revenue_plot,
    build_units_sold_distribution_plot,
    build_price_vs_units_sold_plot,
)

sales_df = load_sales_data()
sales_df.head()

If the dataset cannot be located, ensure `sales_data.csv` from Day 24 resides in the Day 27 directory or adjust `load_sales_data` to point to the correct path.

## 2. Visualise core business questions
Each section reuses the pure functions so the same logic powers both the script and this notebook.

### Revenue by Region
We aggregate revenue by sales region to understand geographic performance.

In [ ]:
if sales_df.empty:
    raise RuntimeError("Sales dataset not available for plotting.")

build_revenue_by_region_plot(sales_df)

### Daily Revenue Trend
Tracking total revenue by day highlights seasonality and the impact of campaigns.

In [ ]:
build_daily_revenue_plot(sales_df)

### Units Sold Distribution
A histogram reveals the spread of transaction sizes and potential outliers.

In [ ]:
build_units_sold_distribution_plot(sales_df)

### Price vs. Units Sold
The scatter plot pairs pricing with units sold to surface product-level dynamics.

In [ ]:
build_price_vs_units_sold_plot(sales_df)

## 3. Interpretation checklist
1. **Compare magnitudes:** The bar chart pinpoints high performing regions.
2. **Spot temporal patterns:** The line chart highlights spikes or dips that merit investigation.
3. **Assess distribution:** The histogram uncovers whether most orders cluster around a typical size.
4. **Diagnose relationships:** The scatter plot shows whether price changes influence demand across products.